In [1]:
def get_datas(ano = '2021'):
        ama_datas = glob.glob(f'/home/andre-pereira/projects/data_science/materias/executivo/municipal/ama/edicoes/{ano}/*/*/*.txt')
        maceio_datas = glob.glob(f'/home/andre-pereira/projects/data_science/materias/executivo/municipal/maceio/prefeitura/edicoes/{ano}/*/*/*.txt')

        print(f'the number of txt archive is: {len(ama_datas)}')
        print(f'the number of txt archive is: {len(maceio_datas)}')

        return ama_datas + maceio_datas

In [2]:
def archive_txt_with_path(path):
    file = open(path, "r")
    file = file.read()
    return file    

In [3]:
def dataframe(recived_data):
    if len(recived_data) == 0:
        return 'Your data is empty!'
    datas = pd.DataFrame()
    for path in recived_data:
        file = archive_txt_with_path(path)
#         print(f'path: {path} -  classification: {was_classifield}')
        datas = datas.append({'texto': file}, ignore_index=True)
    
    datas['id_item'] = datas.index
    
    return datas

In [4]:
def data_classification(recived_data):
    if len(recived_data) == 0:
        return 'Your data is empty!'
    datas = pd.DataFrame()
    for path in recived_data:
        file = archive_txt_with_path(path)
        was_classifield =  event.search(file)
#         print(f'path: {path} -  classification: {was_classifield}')
        datas = datas.append({'path': path, 'text': file, 'classification': was_classifield}, ignore_index=True)
    
    datas['status_of_classification'] = datas['classification'].apply(lambda x: 'classified' if x else 'unclassified')
    return datas

In [5]:

# coding=utf-8
import collections
import re


class EventSearcher(object):

    def __init__(self):
        self.events = collections.OrderedDict([
                       ('TORNAR SEM EFEITO', [r'(to.{1,2}ar\s*sem\s*efeito)']),
            ('CONTRATO TEMPORARIO', [r'extrato\s*de\s*contrato\stemporario', r'resolve.*?contrato\stemporario',
                                     '^\s*contrato servidor temporario']),
            ('SUBSTITUICAO', [r'(designar|nomear|convocar).{0,200}?(substituicao|substituir)',
                              r'autorizar a designacao.*impedimento',
                              r'admitir.*exercer.*impedimento',
                              r'substituir [oa]s? servidor'
                              ]),
            ('ABANDONO DE CARGO', [r'abandono\s*de\s*cargo.*?(demitir|demissao|exoner)']),
            ('CONVOCACAO', [r'convocar para posse', 
                            r'resolve.{0,20}?convocar os candidatos', 
                            r'convocar para entrega de documentacao os candidatos nomeados',
                            r'convocar para tomar posse',
                            r'r e s o l v e[\s\S]{0,50}[oa]s? candidat[oa]s? classificad[oa]s?']),
            ('NOMEACAO EM CARGO EFETIVO', [r'nome(ar|ia)[\s\S]{0,100}candidatos?\s*aprovados?',
                          r'nome(ar|ia)[\s\S]{0,50}carater efetivo',                          
                          r'nomear para os? cargos? publicos?',
                          r'nomeacao de candidatos para os cargos',
                          r'nomear na secretaria.{0,80}?para o cargo publico',
                          r'NOMEAR,? em carater efetivo,?']),
            ('NOMEACAO DE CARGO EM COMISSAO LISTA', [r'nomear.*? (exercerem|para o?s? exercicios?) d?os? cargos?.{3,7}comissao',
                                                     r'.omear os servidores abaixo relacionados no exercicio do cargo.{3,7}comissao',
                                                     r'.omear.*?exercicio.{0,8}carg.{3,8}comi']),
            ('NOMEACAO DE CARGO EM COMISSAO', [r'nomear.*?exercer o cargo.{3,7}comissao',
                                               r'nomear.*?(para o?|no).{4,18}cargo,?.{3,6}comissao',
                                               r'\snomear.{0,150}?cargo.{3,7}comissao',
                                               r'^.{0,5}(designar|mear).*?para o? exer.{7,20}co.{0,2}mi.{0,3}?s',
                                               r'autoriza.*?nomeac.*?comissao']),
            ('CESSAO', [r'(\sautorizar|prorrogar).{2,80}?\s+(cessao|cedencia)',
                        r'\sceder [oa]s? servidor']),
            ('EXONERACAO DE CARGO EM COMISSAO LISTA', [r'.xonera.*?dos cargos.{3,7}comissao']),
            ('EXONERACAO DE CARGO EM COMISSAO', [r'(exonera|dispensa).*?car\s*gos?.{1,15}co.{0,2}mi.{0,2}s']),
            ('EXONERACAO DE CARGO ELETIVO', [r'resolve.{0,20}exonerar.{0,20}cargo eletivo']),
            ('EXONERACAO DE CARGO EFETIVO', [r'resolve.{0,150}?exonera.*?servidor',
                                             r'resolve.{0,20}exonera.{0,100}do cargo',
                                            r'EXONERAR,? a pedido,? da?o? servidora? publica?o?']),
            ('APOSENTADORIA', [r'resolve.{1,20}?aposentar', 
                                   r'concede[\s\S]{0,50}aposentadoria (por tempo de contribuicao|compulsoria|por invalidez|voluntaria)',
                                   r'Conceder? o beneficio de aposentadoria',
                                   r'CONCEDER, o beneficio de aposentadoria',
                                   r'concessao do\\n?beneficio de aposentadoria',
                              ]),
            ('ADICIONAL DE INSALUBRIDADE OU PERICULOSIDADE', [r'adicional\s*(de)?\s*(periculosidade|insalubridade)']),
            ('ADICIONAL DE TEMPO DE SERVICO', [r'adicio\s*nal.{2,8}tempo.{4,6}servico']),
            ('ADICIONAL NOTURNO', [r'adicional\s*noturno']),
            ('AFASTAMENTO PARA ESTUDO OU MISSAO NO EXTERIOR', [r'afastamento\s*para\s*estudo\s*no\s*exterior']),
            ('AFASTAMENTO DO PAIS', [r'afastamento\s*do\s*pais']),
            ('AFASTAMENTO PARA QUALIFICACAO PROFISSIONAL', [r'conceder.{0,100}afastamento.{0,50} para qualificacao profissional']),
            ('AFASTAMENTO SEM ONUS', [r'(in)?deferir.{0,100}afastamento sem onus']),
            ('AFASTAMENTO COM ONUS', [r'(in)?deferir.{0,100}afastamento com onus']),
            ('AFASTAMENTO PREVENTIVO', [r'(evento.|determinar|ordenar)\s*o?\s*afastamento\s*preventi(v|n)o']),
            ('AJUDA DE CUSTO', [r'(faz\s*jus\s*a|conceder)\s*ajuda\s*de\s*custo']),
            ('PENSAO POR MORTE', [r'conceder pensao por morte', r'conceder pensao vitalicia', r'concessao do beneficio de pensao por morte']),
            ('ATO NULO', [r'(tor?nar\s*nulo[\s\S]{0,30}decreto|resolve\s*declarar\s*nulo)']),
            ('AVERBACAO DE TEMPO DE CONTRIBUICAO', [r'((r\s*e\s*s\s*o\s*l\s*v\s*e|deferido)[\s\S]{0,50}pedido\s*de\s*averbacao|averbe-se|averbar o tempo)',
                                                    r'conceder?[\s\S]{0,150}averbacao de tempo de contribuicao',
                                                    r'(deferi|conceder)[\s\S]{0,150}averbacao',
                                                    r'averbar.{2,30}?para fins de aposentadoria']),
            ('ALTERACAO JORNADA DE TRABALHO', [r'(r\s*e\s*s\s*o\s*l\s*v\s*e|decreta)[\s\S]{0,50}regime\s*de\s*trabalho',
                                     r'(in)?deferir.{0,50}?alteracao de jornada laboral',
                                     r'(in)?deferir.{0,50}mudanca de carga horaria',
                                     r'(in)?deferir reducao especial de jornada de trabalho', r'alteracao de jornada de trabalho', r'alteracao de carga horaria']),
            ('CASSACAO DE APOSENTADORIA', [r'r\s*e\s*s\s*o\s*l\s*v\s*e[\s\S]{0,50}cassar\s*a\s*aposentadoria']),
            ('CASSACAO DE DISPONIBILIDADE', [r'r\s*e\s*s\s*o\s*l\s*v\s*e[\s\S]{0,50}\scessar\s*a\s*disponibilidade']),
            ('CESSAR OS EFEITOS', [r'r\s*e\s*s\s*o\s*l\s*v\s*e[\s\S]{0,50}(tornar sem|cessar)[\s\S]{0,50}efeito']),
            ('DEMISSAO', [r'r\s*e\s*s\s*o\s*l\s*v\s*e[\s\S]{0,50}demitir']),
            ('DISPENSA', [r'dispensar.{0,150}cargo|cancelar[\s\S]{0,300}distrato|distrato\s*de\s*contrato']),
            ('DISPENSA DO TRABALHO', [r'conceder.{0,150}dispensa do trabalho']),
            ('DISPENSA FUNCAO GRATIFICADA', [r'dispensar.{0,150}funcao gratificada']),
            ('DESIGNACAO FUNCAO GRATIFICADA', [r'esignar.{0,150}funcao gratificada', 
                                            r'(designar|nomear).{0,120}? (para exercer a|na|para a) funcao',
                                            r'nomear\s*.{0,60}?para exercer o? cargo de']),
            ('PAGAMENTO GRATIFICACAO', [r'pagamento de gratificacao']),
            ('PAGAMENTO HORAS EXCEDENTES', [r'pagamento de horas excedentes']),
            ('DISPONIBILIDADE', [r'(colocar|prorrogar).{0,150}dispo(sicao|nibilidade)']),
            ('ENQUADRAMENTO', [r'ficam?[\s\S]{0,100}enquadrad(a|o|os)', r'.nquadrar.{0,150}cargo', r'enquadramento dos servidores', r'revisao d[eo] enquadramento']),
            ('ESTABILIDADE CONSTITUCIONAL', [r'resolve[\s\S]{0,150}(conceder\s*estabilidade|declarar[\s\S]{0,100}estavel)[\s\S]{0,250}art']),
            ('ESTABILIDADE FINANCEIRA', [r'resolve.{0,150}?estabilidade financeira']),
            ('LICENCA CAPACITACAO', [r'(in)?deferir.{0,50}licenca (para)? capacitacao']),
            ('ESCALA DE FERIAS', [r'(resolve|aprovar)[\s\S]{0,150}escala\s*de\s*ferias']),
            ('LICENCA A ADOTANTE', [r'(resolve|conceder)[\s\S]{0,150}licenca\s*a?\s*adotante']),
            ('LICENCA A PATERNIDADE', [r'(resolve|conceder)[\s\S]{0,180}licenca\s*paternidade']),
            ('LICENCA A GESTANTE', [r'resolve[\s\S]{0,50}licenca\s*a?\s*gestante',
                                    r'conceder?[\s\S]{0,190}lic.{5,9}mat.{1,5}nidade',
                                    r'licenca gestante', 'lic.{2,9}maternidade']),
            ('LICENCA PARA QUALIFICACAO PROFISSIONAL', [r'(resolve|conceder?)[\s\S]{0,150}licenca\s*para\s*qualificacao\s*profissional']),
            ('LICENCA PARA ATIVIDADE POLITICA', [r'resolve[\s\S]{0,300}(atividade\s*politica|mandato\s*eletivo)']),
            ('LICENCA PARA DESEMPENHO DE MANDATO CLASSISTA', [r'resolve[\s\S]{0,250}mandato\s*classista']),
            ('LICENCA PARA QUALIFICACAO PROFISSIONAL', [r'resolve[\s\S]{0,350}licenca\s*para\s*qualificacao\s*profissional']),
            ('LICENCA PARA TRATAMENTO DE SAUDE EM PESSOA DA FAMILIA', [r'(resolve|conceder)[\s\S]{0,150}licenca\s*para\s*tratamento.{0,150}familia',
                                                                       r'conceder[\s\S]{0,150}em pessoa da familia']),
            ('LICENCA PARA TRATAMENTO DE SAUDE', [r'(resolve|concede)[\s\S]{0,150}para\s*tratamento.{2,5}saude',
                                                  r'(resolve|concede|(in)?deferir).{0,150}licenca medica',
                                                  r'atestado medico emitido pelo']),
            ('LICENCA PARA TRATAR DE INTERESSE PARTICULAR', [r'(in)?deferir.{0,150}?(licenca|afastamento).{0,20}?\s*para\s*trata(r|mento)\s*de\s*(interesse|assunto)s?\s*particular(es)?']),
            ('LICENCA POR ACIDENTE EM SERVICO', [r'resolve[\s\S]{0,150}licenca\s*por\s*acidente\s*em\s*servico']),
            ('LICENCA PREMIO', [r'(resolve|conceder?)[\s\S]{0,150}icenca\s*-?\s*premio',
                                r'conceder[\s\S]{0,150}lic[\s\S]{0,50}assidui',
                                r'conceder[\s\S]{0,150}premio[\s\S]{0,50}assidui',
                                r'conceder[\s\S]{0,150}assidui',
                                r'((in)?DEFERIR?|conceder?).{0,50}?Licenca Premio']),
            ('LICENCA POR FALECIMENTO', [r'conceder?[\s\S]{0,200}falecimento',]),
            ('LICENCA POR MOTIVO DE AFASTAMENTO DO CONJUGE', [r'(conceder|(in)?deferir).{0,250}LICENCA POR MOTIVO DE AFASTAMENTO DO CONJUGE']),
            ('LICENCA ESPECIAL', [r'conceder?[\s\S]{0,150}licenca especial']),
            ('LICENCA', [r'conceder?[\s\S]{0,150}?licenca']),
            ('LOTACAO', [r'r\s*e\s*s\s*[o0]\s*l.{2,20}lotar']),
            ('PROGRESSAO', [r'(resolve|conceder?).{0,150}?(progressao|ascencao)', r'lancamento das progressoes']),
            ('PROMOCAO', [r'resolve[\s\S]{0,50}promover']),
            ('REDISTRIBUICAO', [r'resolve[\s\S]{0,50}redistribuir']),
            ('REMOCAO', [r'resolve[\s\S]{0,50}remover']),
            ('REVERSAO', [r'reverter[\s\S]{0,50}servico\s*publico']),
            ('SALARIO-FAMILIA', [r'(conceder|deferido)[\s\S]{0,50}salario\s*familia']),
            ('DESIGNACAO DE FUNCAO', [r'designar [oa]? servidora?.{0,40}?funcao']),
            ('DESIGNACAO', [r'designar[\s\S]{0,80}servidor',
                            r'designar?.{0,200}(para|matricula)',
                            r'RESOLVE.{2,20}\n(designar.{0,150}?como responsaveis)',
                            r'designar.{0,50}para responder pelo cargo',
                            r'prorrogar a designacao',
                            r'r e s o l v e.{0,50}d e s i g n a r']),
            ('DELEGACAO', [r'delegar?.{0,80}servidor']),
            ('SUSPENSAO', [r'suspender[\s\S]{0,50}servidor',
                           r'r\s*e\s*s\s*o\s*l\s*v\s*e.{0,100}?suspender']),
            ('TRANSPOSICAO DE CARGO', [r'resolve[\s\S]{0,50}transpor\s']),
            ('TRANSFERENCIA', [r'resolve.{0,50}transferir', 'trans.eri.{0,5}[oa] servidor']),
            ('FERIAS', [r'conceder[\s\S]{0,150}ferias', r'escala de ferias']),            
            ('INCORPORACAO', [r'conceder[\s\S]{0,200}incorporacao de verba de representacao', r'conceder[\s\S]{0,200}incorporacao']),
            ('VANTAGENS FINANCEIRAS', [r'conceder[\s\S]{0,200}vantage(m|ns) financeiras?']),
            ('INCORPORACAO SALARIAL', [r'conceder[\s\S]{0,200}incorporacao salarial']),
            ('VACANCIA', [r'r e s o l v e[\s\S]{0,50}declarar a vacancia',
                          r'(\s(in)?deferir|conceder).{0,50}vacancia']),
            ('ESTABILIDADE', [r'fica declarado estavel no servico publico',
                              r'estavel no servico publico',
                              r'conceder[\s\S]{0,150}(estabilidade|declarar[\s\S]{0,100}estavel)',
                              r'(declarar|considerar) estave(l|is) no servico publico']),
            ('GRATIFICACAO ADICIONAL', [r'conceder.*?gratificacao']),
            ('ADICIONAL DE QUALIFICACAO', [r'(conceder|(in)?deferir).*adicional de qualificacao']),
            ('GRATIFICACAO REPRESENTACAO', [r'(concede|atribui).*gratificacao.{0,6}representacao']),
            ('ABONO DE PERMANENCIA', [r'(conceder|(in)?deferir).{1,50}?abono de permanencia']),
            ('REENQUADRAMENTO', [r'reenquadrar os? servidor(es)?', r'reenquadramento (para|na)', r'promocao para']),
            ('REENQUADRAMENTO', [r'reenquadrar os? servidor(es)?', r'reenquadramento (para|na)', r'promocao para']),
            ('RECONDUCAO', [r'reconduzir.{0,50}para']),
            ('ELEVACAO DE NIVEL', [r'\s(in)?deferir.{0,50}elevacao de nivel', r'concede mudanca de nivel', ]),
            ('RETORNO', [r'retornar.{1,100}[oa]s servidor',
                         r'fazer retornar as (suas)? atividades (normais)?',
                         r'retorno as atividades',
                         r'resolve.{2,20}? fazer retornar',
                         r'resolve.{2,30}?retornar']),
            ('APOSTILAMENTO', [r'\s(conceder|(in)?deferir|cancelar).{0,150}apostilamento']),
            ('NOMEAR INTERINAMENTE', [r'NOMEAR INTERINAMENTE']),
            ('READAPTACAO DE FUNCAO', [r'conceder Readaptacao de Funcao',
                                       r'readaptar em funcao',
                                       r'autorizar a? readaptacao de funcao',]),
            ('DEMISSAO', [r'resolve.{0,10}?(demissao|demitir)']),
            ('DESTITUIR', [r'resolve.{0,10}?(destituir)']),
            ('ELEVACAO DE CLASSE', [r'[\s-](in)?defer?ir.{0,50}?elevacao de classe?.{0,10}?servidor']),
            ('DEDICACAO EXCLUSIVA', [r'\s((in)?deferir|conceder).{0,50}?dedicacao exclusiva']),
            ('RESCISAO CONTRATRUAL', [r'\s(resolve).{0,50}?Rescindir o Contrato', 'rescindir a pedido']),
            ('CONTRATO TEMPORARIO', [r'C O N T R A T O (S E R V I D O R)? T E M P O R A R I O']),
            ('ELEVACAO FUNCIONAL', [r'conceder ascensao funcional']),
            ('RECLASSIFICAR', [r'reclassifica']),
            ('PROCESSO DISCIPLINAR', [r'(o|instauracao de|instaurar) processo administrativo disciplinar']),
            ('ADVERTENCIA DISCIPLINAR', [r'aplicar.{0,50}?advertencia disciplinar']),
            ('HORAS EXCEDENTES', [r'conceder horas excedentes']),
            ('AUXILIO DOENCA', [r'auxilio doenca aos servidores']),            
            ('HOMOLOGACAO DE ESTAGIO PROBATORIO', [r'avaliacao especial de desempenho de estagio probatorio dos servidores']),
            ('SALARIO FAMILIA', [r'concessao (do beneficio)? de salario[\s-]familia']),            
            ('DESIGNACAO PARA COMISSAO/CONSELHO/FISCAL/GRUPO', [
                r'ficam nomeados (como|os) membros.{0,70}?(conselho|junta)',
                r'ficam nomeados.{0,40}?para compor(em)? o conselho',
                r'fica criada e nomeada a comissao',
                r'instituir composicao atualizada da comissao permanente',
                r'servidores abaixo discriminados, como gestor, fiscal titular e fiscal suplente nos contratos',
                r'(nomear|constituir) comissao para',
                r'constituir comissao (disciplinar|de\s|composta|temporaria|permanente)',
                r'instituir a? comissao (destinada|permanente|temporaria)',
                r'(alterar|atualizar) a? composicao da comissao',
                r'constituir a subcomissao',
                r'dar posse aos conselheiros',
                r'criar a comissao organizadora',
                r'homologar a composicao do conselho',
                r'passa a? compor a comissao',
                r'aprovar a indicacao dos conselheiros.{0,40}?comissao'
                ]),
            ('RETIFICACAO', [r'(retificar|retifica\s)', r'(in)?deferir o? pedido de retificacao']),
        ])

        self.events_regex = {}

        for event in self.events.keys():
            self.events_regex[event] = []

            for regex in self.events[event]:
                self.events_regex[event].append(re.compile(regex, re.DOTALL | re.MULTILINE | re.IGNORECASE))

    def search(self, snippet):
        for event in self.events.keys():
            for keyword in self.events[event]:
                if keyword in snippet:
                    return event

            for regex in self.events_regex[event]:
                match = regex.search(snippet)
                if match is not None:
                    return event

        return None

In [6]:
event = EventSearcher()
def regex(df):
    datas = pd.DataFrame()
    for idx, row in df.iterrows():
        text = row['texto']
        was_classifield =  event.search(text)
#         print(f'path: {path} -  classification: {was_classifield}')
        datas = datas.append({'texto': text, 'label_regex': was_classifield}, ignore_index=True)
    
    datas['status'] = datas['label_regex'].apply(lambda x: 'classified' if x else 'unclassified')
    return datas

In [7]:
def filtra_dataset(df, classification_list):
    # Filtra dataset
    df = df[df.status == 'classified']
    outros = df.query("label_regex not in @classification_list & status == 'classified'")
    outros['label_regex'] = 'OUTROS'
    df_lista = df.query("label_regex in @classification_list")
    # retorna dataset
    df_resultado = pd.concat([df_lista, outros])
    # df_resultado.reset_index()
    return df_resultado

In [8]:
# Vetorização TD-IDF e pipeline

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

#classificadores
from sklearn.linear_model import SGDClassifier

def pipeline_vetorizacao_classificacao():
    return Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', SGDClassifier(loss='modified_huber')),
                ])

In [9]:
import rubrix as rb

def envio_rubrix(dataset, pipeline, n_versao, agent_name='nome do agente'):
        rb.init(api_url="http://localhost:6900")

        # Build the Rubrix records
        records = [
            rb.TextClassificationRecord(
                id=idx,
                inputs={'id': row['id_item'], 'texto': row['texto']},
                prediction=list(zip(pipeline.classes_, row['probabilities'])),
                prediction_agent=agent_name,

            )
            for idx, row in dataset.iterrows()
        ]

        # Log the records
        rb.log(records, name=f'erros_rubrix_v{n_versao}', 
            tags={
                "dataset": "erros classificacao diarios oficiais 2021",
                "metodologia": "dados off diagonal da matriz de confusao e com probabilidade menor que 90%"
                },
            )


2022-04-26 05:18:23.098793: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-26 05:18:23.098846: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/andre-pereira/.local/lib/python3.8/site-packages/jose/backends/cryptography_backend.py:18: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes, int_to_bytes


In [10]:
from sklearn.model_selection import train_test_split

def train_dataset(pipeline, df, x, y, n_versao):

    x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.5, random_state=42, shuffle=True, stratify=y)
    pipeline.fit(x_train, y_train)
    print(pipeline.predict(x_test))
    print(pipeline.predict_proba(x_test))
    print(pipeline.score(x_test, y_test))
    y_pred_all = pipeline.predict(x)
    df[f'predict_label_v{n_versao}'] = y_pred_all
    y_pred_proba_all = pipeline.predict_proba(x)
    df[f'predict_proba_label_v{n_versao}'] = list(y_pred_proba_all)
    
    joblib.dump(pipeline, f'./versiona_vida_funcional_sgd/pipeline/sgd_pipeline_v{n_versao}.pkl')
    df.to_pickle(f'./versiona_vida_funcional_sgd/dataset/dataset_treinado_v{n_versao}.pkl')
    
    return df, pipeline


In [11]:
# merge

def merge_dataset(df, df2, n_versao):
    # listas de id's
    lists_id = [idx for idx in df2.id_item]
    df[f'merge_label_v{n_versao}'] = df[f'predict_label_v{n_versao}']
  
    print(len(df))
    for idx, row in df.iterrows():
        id_item = row['id_item']
        if id_item in lists_id:
            df.loc[df.id_item == id_item, f'merge_label_v{n_versao}'] = df2['label'][df2.id_item == id_item].values[0]
    print(len(df))
    filtra = list(df2.id_item.apply(int))
    print(df.query('id_item in @filtra').head(30))
    df.to_pickle(f'./versiona_vida_funcional_sgd/dataset/dataset_merge_v{n_versao}.pkl')
    return df

In [12]:
def extrai_inputs_rubrix(df_rubrix, n_versao):
    df = None
    df = pd.DataFrame()
    for t, row in df_rubrix.iterrows():
        df = df.append({'id_item': int(float(row['inputs']['id'])), 'label': row['annotation'], 'texto': row['inputs']['texto']}, ignore_index=True)
#     print(row['id'], row['annotation'], row['inputs']['text'])
    df.to_pickle(f'./versiona_vida_funcional_sgd/dataset/df_correcao_rubrix_v{n_versao}.pkl')
    df.id_item = df.id_item.apply(int)
    return df

In [13]:
def load_df_rubrix(nome_rubrix, n_versao):
    rb.init(api_url="http://localhost:6900")

    df = rb.load(nome_rubrix)
    
    df = extrai_inputs_rubrix(df, n_versao)
    
    return df

In [14]:
import pandas as pd

def treina_e_envia_erros(n_nova_versao, n_antiga_versao):
    pipeline_antigo = joblib.load(f'./versiona_vida_funcional_sgd/pipeline/sgd_pipeline_v{n_antiga_versao}.pkl')

    dataset_merge_antigo = pd.read_pickle(f'./versiona_vida_funcional_sgd/dataset/dataset_merge_v{n_antiga_versao}.pkl')
    
    x = dataset_merge_antigo.texto
    y = dataset_merge_antigo[f'merge_label_v{n_antiga_versao}']
    
    dataset_treinado_novo, pipeline_novo = train_dataset(pipeline_antigo, dataset_merge_antigo, x, y, n_nova_versao)
    
    df_erros = extraindo_erros_dataset(dataset_treinado_novo, pipeline_novo, n_antiga_versao, n_nova_versao)


    envio_rubrix(df_erros, pipeline_novo, n_nova_versao,'SGDClassifier')
    return dataset_treinado_novo

In [15]:
def recebe_dados_rubrix_e_faz_merge(df, n_nova_versao):
    df_corrigido = load_df_rubrix(f'erros_rubrix_v{n_nova_versao}', n_nova_versao)
    
    dataset_merge_novo = merge_dataset(df, df_corrigido, n_nova_versao)
    return dataset_merge_novo

In [16]:
def extraindo_erros_dataset(df, pipeline, n_antiga_versao, n_nova_versao):
        df2 = df[df[f'merge_label_v{n_antiga_versao}'] != df[f'predict_label_v{n_nova_versao}']]
        erros = df2[df2[f'predict_proba_label_v{n_nova_versao}'].apply(max) < 0.9]

        erros_prob = pipeline.predict_proba(erros.texto)
        
        print('A quantidade de dados classificados com erro neste treinamento é: {} '.format(len(erros)))
        erros['probabilities'] = list(erros_prob)
        return erros

In [17]:
def predict_all_datas_ml(pipeline):
    # extrai materias
    df =  get_datas()
    df = dataframe(df)
    y_pred_all = pipeline.predict(df.texto)
    df['predict_classification'] = y_pred_all

    y_pred_proba_all = pipeline.predict_proba(df.texto)

    df['predict_proba_label'] = list(y_pred_proba_all)

    df.to_pickle('./versiona_vida_funcional_sgd/dataset/predict_all_datas_ml.pkl')
    return df